In [2]:
# load libraries and file
import numpy as np
import scipy.signal as sigp
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from ipywidgets import interact, interactive, fixed, interact_manual,Checkbox,Layout,Box,VBox,Label
import ipywidgets as widgets
from sklearn.cluster import KMeans
# import mpld3
# mpld3.enable_notebook()
# plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.figsize'] = [9, 6]
%matplotlib widget
data = sio.loadmat("channel27.mat")['ch27']
sampling_frequency=25e+3
segment_view_start=0 #in ms
segment_view_end=6000
segment_view_start=np.argmin(data[:,0]<=segment_view_start)#in time points
segment_view_end=np.argmin(data[:,0]<=segment_view_end)


In [3]:
# view the data

plt.close()
plt.figure()

plt.plot(data[segment_view_start:segment_view_end,0],data[segment_view_start:segment_view_end,1]);
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
#find spikes and plot the peaks eyeballing

global_threshold = 0
plt.close()

is_peaks_maximum=False

p_sign=is_peaks_maximum*2-1

def plot_peaks(n):

    print(n)
    plt.close()
    threshold = np.median(np.abs(p_sign*data[:,1]-np.median(data[:,1])))*n
    print(threshold)
    segmented_data=data[segment_view_start:segment_view_end,:]
    peaks,_ = sigp.find_peaks(p_sign*segmented_data[:,1],height=threshold)
    plt.plot(segmented_data[:,0],segmented_data[:,1]);
    peaks_to_view_mask=peaks<segment_view_end
    peaks_to_view_mask=peaks_to_view_mask>=segment_view_start
    plt.scatter(segmented_data[peaks,0],segmented_data[peaks,1],color='red')
    global_threshold = threshold
    plt.show()
_=interact(plot_peaks, n = widgets.FloatSlider(value=4,
                                               min=1,
                                               max=25,
                                               step=0.5))



interactive(children=(FloatSlider(value=4.0, description='n', max=25.0, min=1.0, step=0.5), Output()), _dom_cl…

In [5]:
n=17
plt.close()
plot_peaks(n)
plt.show()

17
88.57


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# find peaks and segment
def find_peaks_and_segment(threshold_factor_in_md,start_in_ms,end_in_ms,is_peaks_maximum=False):


    p_sign=is_peaks_maximum*2-1
    threshold = np.median(np.abs(data[:,1]-np.median(data[:,1])))*threshold_factor_in_md
    peaks,_ = sigp.find_peaks(p_sign*data[:,1],height=threshold)

    spike_traces_windows_size_ms =np.array([start_in_ms,end_in_ms])#the window in ms

    spike_traces_windows_size = spike_traces_windows_size_ms*(sampling_frequency/1000)
    spike_traces_windows_size[0],spike_traces_windows_size[1]=np.floor(spike_traces_windows_size[0]),np.ceil(spike_traces_windows_size[1])
    spike_traces_windows_size = spike_traces_windows_size.astype(int)
    print(spike_traces_windows_size)
    spikes = []

    for peak in peaks:
        if peak<spike_traces_windows_size[0] or peak>data.shape[0]-spike_traces_windows_size[1]: #leave those ones in the edges.
            continue

        spikes.append(data[peak+spike_traces_windows_size[0]:peak+spike_traces_windows_size[1],1]) #align the peak to the center
    spikes = np.vstack(spikes)
    return spikes
spike_traces_windows_size_ms=[-1,3]
spikes = find_peaks_and_segment(17,*spike_traces_windows_size_ms)

[-25  75]


In [8]:
#time validation
validation_spikes = find_peaks_and_segment(17,-1,3)
plt.close()
fig, (ax1) = plt.subplots(1, 1)
ax1.plot(np.linspace(-0.75,3,validation_spikes.shape[1]),validation_spikes.T)
plt.show()
del validation_spikes

[-25  75]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#looking at our spikes
plt.close()
fig, (ax1) = plt.subplots(1, 1)
ax1.plot(np.linspace(spike_traces_windows_size_ms[0],spike_traces_windows_size_ms[1],spikes.shape[1]),spikes.T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.close()
fig, (ax1) = plt.subplots(1, 1)
s_mean = np.mean(spikes.T,axis=1)
s_std=np.std(spikes.T,axis=1)
ax1.plot(np.linspace(spike_traces_windows_size_ms[0],spike_traces_windows_size_ms[1],spikes.shape[1]),s_mean)
a=s_mean-s_std
ax1.fill_between(np.linspace(spike_traces_windows_size_ms[0],spike_traces_windows_size_ms[1],spikes.shape[1]),s_mean-s_std,s_mean+s_std,alpha=.1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# remove mean.
spike_mean = np.mean(spikes,axis=1)
baslined_spikes=spikes-spike_mean[...,np.newaxis]
plt.close()
plt.figure()
plt.plot(np.linspace(spike_traces_windows_size_ms[0],spike_traces_windows_size_ms[1],spikes.shape[1]),baslined_spikes.T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
#correlations
plt.close()
fix,ax= plt.subplots()
corr = np.corrcoef(baslined_spikes.T)
im = ax.matshow(corr,cmap='seismic',vmin=-1, vmax=1,origin='lower')
# plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=45)
# plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14).colorbar()
# cb.ax.tick_params(labelsize=14)
points_in_ms=np.array([-1,-0.5,0,0.5,1,1.5,2,2.5,3])
points_in_sampling=points_in_ms*sampling_frequency/1000
points_in_sampling-=points_in_sampling[0]
points_in_sampling[-1]-=1
ax.xaxis.tick_bottom()
ax.set_xticks(points_in_sampling)
ax.set_xticklabels(points_in_ms)
ax.set_yticks(points_in_sampling)
ax.set_yticklabels(points_in_ms)
ax.set_xlabel("time(ms)")
ax.set_ylabel("time(ms)")
plt.title("corrlation in time")
_ = plt.colorbar(im)
# ax.title('Correlation Matrix', fontsize=16);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# singular value decomposition
plt.cla()
plt.close()
u, s, vh = np.linalg.svd(baslined_spikes.T)
directions_normlized = u;
standard_deviations = np.diag(s);
coefficients = vh;
fig ,axs =  plt.subplots(3, 2)
axs[0,0].stem(s)
axs[0,0].set_xlabel("Pc number")
axs[0,0].set_ylabel("variation factor")

for i in range(1,6):
    for j in range(2):
        axs[i//2,i%2].plot(np.linspace(spike_traces_windows_size_ms[0],spike_traces_windows_size_ms[1],baslined_spikes.shape[1]),directions_normlized[:,i*2-2+j],label="Pc %d"%(i*2-2+j))
    axs[i//2,i%2].set_xlabel("time(ms)")
    axs[i//2,i%2].legend()
# plt.legend()
_=plt.tight_layout()
_=plt.show



1.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\python\lib\site-packages\ipykernel_launcher.py:10: UserWarning: In Matplotlib 3.3 individual lines on a stem plot will be added as a LineCollection instead of individual lines. This significantly improves the performance of a stem plot. To remove this warning and switch to the new behaviour, set the "use_line_collection" keyword argument to True.
  # Remove the CWD from sys.path while we load stuff.


In [14]:
#another way to view the results
us = u*s
fig, ax = plt.subplots()
im = ax.matshow(u.T,cmap='hot')
print(u.shape)
ax.vlines(0*sampling_frequency/1000+1*sampling_frequency/1000,ymin=0,ymax=u.shape[0]-1,color='blue')
points_in_ms=np.array([-1,-0.5,0,0.5,1,1.5,2,2.5,3])
points_in_sampling=points_in_ms*sampling_frequency/1000
points_in_sampling-=points_in_sampling[0]
points_in_sampling[-1]-=1
ax.xaxis.tick_bottom()
ax.set_xticks(points_in_sampling)
ax.set_xticklabels(points_in_ms)
ax.set_ylabel("component")
ax.set_xlabel("time")
_ = plt.colorbar(im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(100, 100)


In [15]:

singular_values_to_keep=[]
singular_values_to_remove=[]

singular_values_to_keep=np.array(singular_values_to_keep).astype(int)
singular_values_to_remove=np.array(singular_values_to_remove).astype(int)
if len(singular_values_to_keep)==0:
    singular_values_to_keep=np.arange(u.shape[1])
if len(singular_values_to_remove)!=0:
    singular_values_to_keep = singular_values_to_keep[~np.isin(singular_values_to_keep,singular_values_to_remove)]
# singular_values_to_keep = singular_values_to_keep[sorted(values_to_keep.values(),key=values_to_keep.keys())]


removed_standard_deviations = np.zeros((u.shape[1],vh.shape[0]))
mask = np.zeros_like(removed_standard_deviations)
mask[singular_values_to_keep,singular_values_to_keep]=1
mask = mask.astype(np.bool)
plt.close()
plt.figure()
removed_standard_deviations[:standard_deviations.shape[0],:standard_deviations.shape[1]] = standard_deviations[:,:]
removed_standard_deviations[~mask]=0
fixed_data=u@removed_standard_deviations@vh
plt.plot(np.linspace(spike_traces_windows_size_ms[0],spike_traces_windows_size_ms[1],spikes.shape[1]),fixed_data)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
#correlations
plt.close()
fix,ax= plt.subplots()
corr = np.corrcoef(fixed_data)
im = ax.matshow(corr,cmap='seismic',vmin=-1, vmax=1,origin='lower')
# plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=45)
# plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14).colorbar()
# cb.ax.tick_params(labelsize=14)
points_in_ms=np.array([-1,-0.5,0,0.5,1,1.5,2,2.5,3])
points_in_sampling=points_in_ms*sampling_frequency/1000
points_in_sampling-=points_in_sampling[0]
points_in_sampling[-1]-=1
ax.xaxis.tick_bottom()
ax.set_xticks(points_in_sampling)
ax.set_xticklabels(points_in_ms)
ax.set_yticks(points_in_sampling)
ax.set_yticklabels(points_in_ms)
ax.set_xlabel("time(ms)")
ax.set_ylabel("time(ms)")
plt.title("corrlation in time")
_ = plt.colorbar(im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:

def clustering(n):
    plt.cla()
    plt.close('all')

    fig,ax = plt.subplots(int(n//2 + (n%2!=0)),2)

    # ax = fig.add_subplot(projection='3d')
    clustering = KMeans(n_clusters=n,random_state=0).fit(baslined_spikes)
    labels = clustering.labels_
    for i in range(n):
    #     ax.scatter(data_3d[0,labels==i],data_3d[1,labels==i],data_3d[2,labels==i])
        if int(n//2 + (n%2!=0))==1:
               ax[i%2].plot(baslined_spikes.T[:,labels==i])
               continue
        ax[i//2,i%2].plot(baslined_spikes.T[:,labels==i])
    plt.tight_layout()
    plt.show()


In [18]:
plt.close()
clustering(3)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …